Решающее дерево бинарной классификации на основе Индекса Джини и выбором одного из sqrt(N) рандомных сплитов. Каждый узел реализовал как объект класса, mean это по факту предиктор (показывает, какой класс преобладает в области).

+ Киллер-фича: дополнительный штраф за выбор сплита с большим дисбалансом в размере дочерних узлов <br>`imbalance = abs(left_mask.sum() - len(X)/2) / len(X)`

киллер фича улучшает качество, т.к. если поддерево почти не отличается от родительского узла(или наоборот, слишком маленькое (шумное)) → разделение бесполезное, но увеличивает глубину дерева.

In [1]:
import numpy as np

In [2]:
class MyDecisionTree:
    def __init__(self, max_depth=5, min_samples=5):
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.root = None

    class Node:
        def __init__(self, mean, split_feature=None, split_value=None):
            self.mean = mean
            self.split_feature = split_feature
            self.split_value = split_value
            self.left = None
            self.right = None

    def _gini(self, y_left, y_right):
        p_left = y_left.mean()
        p_right = y_right.mean()
        return 2*p_left*(1-p_left) + 2*p_right*(1-p_right)

    def _best_split(self, X, y):
        best = (None, None, np.inf)
        
        for p in range(X.shape[1]):
            values = np.unique(X[:, p])
            
            for v in values:
                left_mask = X[:, p] < v
                if left_mask.sum() < self.min_samples or (~left_mask).sum() < self.min_samples:
                    continue
                
                gini = self._gini(y[left_mask], y[~left_mask])
                imbalance = abs(left_mask.sum() - len(X)/2) / len(X)
                score = gini + 0.5*imbalance
                
                if score < best[2]:
                    best = (p, v, score)
        
        return best

    def _build_tree(self, X, y, depth):
        node = self.Node(y.mean())
        
        if depth == 0 or len(y) < 2*self.min_samples:
            return node
            
        p, v, _ = self._best_split(X, y)
        if p is None:
            return node
            
        left_mask = X[:, p] < v
        node.split_feature = p
        node.split_value = v
        node.left = self._build_tree(X[left_mask], y[left_mask], depth-1)
        node.right = self._build_tree(X[~left_mask], y[~left_mask], depth-1)
        
        return node

    def fit(self, X, y):
        self.root = self._build_tree(X, y, self.max_depth)

    def _predict(self, x, node):
        if node.split_feature is None:
            return node.mean
        if x[node.split_feature] < node.split_value:
            return self._predict(x, node.left)
        return self._predict(x, node.right)

    def predict(self, X):
        return np.array([self._predict(x, self.root) for x in X])

In [3]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

np.random.seed(42)
dataset = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, train_size=0.7)

In [4]:
kop = MyDecisionTree(min_samples=5, max_depth=5)
kop.fit(X_train, y_train)
((kop.predict(X_test)>0.5) == y_test).mean()

np.float64(0.935672514619883)

In [5]:
from sklearn.tree import DecisionTreeClassifier

pok = DecisionTreeClassifier(criterion='gini', max_features='sqrt')
pok.fit(X_train, y_train)
(pok.predict(X_test) == y_test).mean()


np.float64(0.935672514619883)